In [110]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

In [111]:
from pandasql import sqldf

In [112]:
# Points to remember
#The main function used in pandasql is sqldf. 
#sqldf accepts 2 parametrs

# a sql query string
# a set of session/environment variables (locals() or globals())


In [113]:
#define the global function, as managing local and global environment might be confusing at times
pysqldf = lambda q: sqldf(q, globals())

In [114]:
pysqldf

<function __main__.<lambda>>

In [115]:
#pandasql uses SQLite syntax. 

#Any pandas dataframes will be automatically detected by pandasql. 

#You can query them as you would any regular SQL table.

In [116]:
from pandasql import sqldf, load_meat, load_births

In [117]:
pysqldf = lambda q: sqldf(q, globals())

In [118]:
meat = load_meat()

In [119]:
meat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827 entries, 0 to 826
Data columns (total 8 columns):
date               827 non-null datetime64[ns]
beef               827 non-null float64
veal               827 non-null float64
pork               827 non-null float64
lamb_and_mutton    827 non-null float64
broilers           635 non-null float64
other_chicken      143 non-null float64
turkey             635 non-null float64
dtypes: datetime64[ns](1), float64(7)
memory usage: 51.8 KB


In [120]:
meat.head()

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01,751.0,85.0,1280.0,89.0,NaN,NaN,NaN
1,1944-02-01,713.0,77.0,1169.0,72.0,NaN,NaN,NaN
2,1944-03-01,741.0,90.0,1128.0,75.0,NaN,NaN,NaN
3,1944-04-01,650.0,89.0,978.0,66.0,NaN,NaN,NaN
4,1944-05-01,681.0,106.0,1029.0,78.0,NaN,NaN,NaN


In [121]:
births = load_births()

In [122]:
births.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 2 columns):
date      408 non-null datetime64[ns]
births    408 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.5 KB


In [123]:
births.head()

,date,births
0,1975-01-01,265775
1,1975-02-01,241045
2,1975-03-01,268849
3,1975-04-01,247455
4,1975-05-01,254545


In [124]:
pysqldf("SELECT * FROM meat LIMIT 5;")

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01 00:00:00.000000,751.0,85.0,1280.0,89.0,None,None,None
1,1944-02-01 00:00:00.000000,713.0,77.0,1169.0,72.0,None,None,None
2,1944-03-01 00:00:00.000000,741.0,90.0,1128.0,75.0,None,None,None
3,1944-04-01 00:00:00.000000,650.0,89.0,978.0,66.0,None,None,None
4,1944-05-01 00:00:00.000000,681.0,106.0,1029.0,78.0,None,None,None


In [125]:
pysqldf("SELECT * FROM births LIMIT 5;")

,date,births
0,1975-01-01 00:00:00.000000,265775
1,1975-02-01 00:00:00.000000,241045
2,1975-03-01 00:00:00.000000,268849
3,1975-04-01 00:00:00.000000,247455
4,1975-05-01 00:00:00.000000,254545


In [126]:
#let's work on the joins and aggregations

In [127]:
# execution of simple group by query
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;"""

In [128]:
print(q)

SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;


In [129]:
pysqldf(q)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0


In [130]:
# you want to print all the results and then do selection of top 10 records
q = """SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year;"""

In [131]:
pysqldf(q).head(10)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0
5,1949,9142.0
6,1950,9248.0
7,1951,8549.0
8,1952,9337.0
9,1953,12055.0


In [132]:
# order by using random functions
q = """SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;"""


In [133]:
print(q)

SELECT 
    *
    FROM
        meat
    ORDER BY RANDOM()
    LIMIT 10;


In [134]:
print (pysqldf(q))

                         date    beef   veal    pork  lamb_and_mutton  \
0  2007-01-01 00:00:00.000000  2166.0   14.3  1898.5             15.0   
1  2011-09-01 00:00:00.000000  2215.2   10.7  1954.4             11.8   
2  1980-03-01 00:00:00.000000  1653.0   30.0  1388.0             28.0   
3  1973-03-01 00:00:00.000000  1774.0   31.0  1134.0             40.0   
4  1974-03-01 00:00:00.000000  1858.0   29.0  1167.0             44.0   
5  1949-07-01 00:00:00.000000   743.0  103.0   584.0             45.0   
6  1978-10-01 00:00:00.000000  2103.0   48.0  1176.0             27.0   
7  1980-10-01 00:00:00.000000  2026.0   38.0  1485.0             29.0   
8  1997-06-01 00:00:00.000000  2132.0   26.0  1312.0             21.0   
9  1953-02-01 00:00:00.000000   849.0   80.0   759.0             57.0   

   broilers  other_chicken  turkey  
0    3029.8           39.5   482.4  
1    3180.4           47.0   473.2  
2     920.0            NaN   127.9  
3     621.4            NaN    45.3  
4     641.9

In [135]:
#executing union all statements
q = """
        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    """

In [136]:
print(q)


        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        
        UNION ALL
        
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    


In [137]:
print (pysqldf(q).head(10))

                         date        meat_type   value
0  1944-01-01 00:00:00.000000             beef   751.0
1  1944-01-01 00:00:00.000000             veal    85.0
2  1944-01-01 00:00:00.000000             pork  1280.0
3  1944-01-01 00:00:00.000000  lamb_and_mutton    89.0
4  1944-02-01 00:00:00.000000             beef   713.0
5  1944-02-01 00:00:00.000000             veal    77.0
6  1944-02-01 00:00:00.000000             pork  1169.0
7  1944-02-01 00:00:00.000000  lamb_and_mutton    72.0
8  1944-03-01 00:00:00.000000             beef   741.0
9  1944-03-01 00:00:00.000000             veal    90.0


In [138]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;
"""

In [139]:
print(q)


    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;



In [140]:
print(pysqldf(q).head())

                         date    beef  veal    pork  lamb_and_mutton  \
0  1975-01-01 00:00:00.000000  2106.0  59.0  1114.0             36.0   
1  1975-02-01 00:00:00.000000  1845.0  50.0   954.0             31.0   
2  1975-03-01 00:00:00.000000  1891.0  57.0   976.0             35.0   
3  1975-04-01 00:00:00.000000  1895.0  60.0  1100.0             34.0   
4  1975-05-01 00:00:00.000000  1849.0  59.0   934.0             31.0   

   broilers  other_chicken  turkey  births  
0     646.2            NaN    64.9  265775  
1     570.2            NaN    47.1  241045  
2     616.6            NaN    54.4  268849  
3     688.3            NaN    68.7  247455  
4     690.1            NaN    81.9  254545  


In [141]:
#Perform inner join on two tables
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;"""

In [142]:
print(q)

SELECT
        m.date, m.beef, b.births
     FROM
        meat m
     INNER JOIN
        births b
           ON m.date = b.date;


In [143]:
#after join, print first 10 records
pysqldf(q).head(10)

,date,beef,births
0,1975-01-01 00:00:00.000000,2106.0,265775
1,1975-02-01 00:00:00.000000,1845.0,241045
2,1975-03-01 00:00:00.000000,1891.0,268849
3,1975-04-01 00:00:00.000000,1895.0,247455
4,1975-05-01 00:00:00.000000,1849.0,254545
5,1975-06-01 00:00:00.000000,1849.0,254096
6,1975-07-01 00:00:00.000000,1916.0,275163
7,1975-08-01 00:00:00.000000,1961.0,281300
8,1975-09-01 00:00:00.000000,2065.0,270738
9,1975-10-01 00:00:00.000000,2270.0,265494


In [144]:
# use queries within queries
q = """
    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)
"""

In [145]:
print(q)


    SELECT 
        m1.date
        , m1.beef 
    FROM 
        meat m1 
    WHERE m1.date IN 
        (SELECT 
            date 
        FROM meat 
        WHERE 
            beef >= broilers 
        ORDER BY date)



In [146]:
pysqldf(q).head()

,date,beef
0,1960-01-01 00:00:00.000000,1196.0
1,1960-02-01 00:00:00.000000,1089.0
2,1960-03-01 00:00:00.000000,1201.0
3,1960-04-01 00:00:00.000000,1066.0
4,1960-05-01 00:00:00.000000,1202.0


In [147]:
from sklearn.datasets import load_iris

In [148]:
import re

In [149]:
iris = load_iris()

In [150]:
#IRIS is now in numpy array format
iris.data[0:5,]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2]])

In [151]:
# convert IRIS to a data frame object
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [152]:
iris_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [153]:
# join the target variable to the iris df
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)

In [154]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [155]:
# replace the '(cm)' in the column header to NULL/blank to clean the feature names
iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]


In [156]:
iris_df.head()

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [157]:
# print first 10 records from the iris df table
print(sqldf("SELECT * FROM iris_df LIMIT 10;"))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            4.9           3.0            1.4           0.2  setosa
2            4.7           3.2            1.3           0.2  setosa
3            4.6           3.1            1.5           0.2  setosa
4            5.0           3.6            1.4           0.2  setosa
5            5.4           3.9            1.7           0.4  setosa
6            4.6           3.4            1.4           0.3  setosa
7            5.0           3.4            1.5           0.2  setosa
8            4.4           2.9            1.4           0.2  setosa
9            4.9           3.1            1.5           0.1  setosa


In [158]:
# select sepal width column and species column and print first 10 records
print(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;"))

   sepalwidthcm species
0           3.5  setosa
1           3.0  setosa
2           3.2  setosa
3           3.1  setosa
4           3.6  setosa
5           3.9  setosa
6           3.4  setosa
7           3.4  setosa
8           2.9  setosa
9           3.1  setosa


In [159]:
# let's compute some statistical functions from the table
# compute the mean, minimum and maximum values of sepal width feature and group by the species category


In [160]:
# the query to perform the above task would look like as below
q = """
      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        
"""

In [161]:
print(q)


      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        



In [162]:
print(sqldf(q))

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.418                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [163]:
#let's create a new table using inner join on the iris df by creating two dummy tables a and b and join by species column
q = """
    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;
"""

In [164]:
print(q)


    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;



In [165]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            5.1           3.5            1.4           0.2  setosa
2            5.1           3.5            1.4           0.2  setosa
3            5.1           3.5            1.4           0.2  setosa
4            5.1           3.5            1.4           0.2  setosa
5            5.1           3.5            1.4           0.2  setosa
6            5.1           3.5            1.4           0.2  setosa
7            5.1           3.5            1.4           0.2  setosa
8            5.1           3.5            1.4           0.2  setosa
9            5.1           3.5            1.4           0.2  setosa


In [166]:
# use a filter criteria to select some records where the condition is met

In [167]:
q = """
    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;
"""

In [168]:
print(q)


    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.7;



In [169]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species
0            7.9           3.8            6.4           2.0  virginica


In [170]:
# create one unique id column in iris df
iris_df['id'] = range(len(iris_df))

In [171]:
iris_df.head()

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species,id
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,2
3,4.6,3.1,1.5,0.2,setosa,3
4,5.0,3.6,1.4,0.2,setosa,4


In [172]:
# fetch the records where a logical condition is met
q = """
    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);
"""

In [173]:
print(q)


    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);



In [174]:
print(pysqldf(q))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species   id
0            5.7           4.4            1.5           0.4     setosa   15
1            7.2           3.6            6.1           2.5  virginica  109
2            7.7           3.8            6.7           2.2  virginica  117
3            7.9           3.8            6.4           2.0  virginica  131
